# Processing FIA DB Files for FNCAP

## Introduction

This document processes FIA data, then sets up initial exploration and visualization for growth modeling.

The next step is growth model estimation.

## 0. Check whether the IDE/.ipynb is working. Get packages.

In [ ]:
# Is the IDE working?

1 + 1

# Instead of learning new things, I'll stick to the Tidyverse. Cheers to the developers.

using Tidier

# Set display rows.

ENV["DATAFRAMES_COLUMNS"] = 10
ENV["DATAFRAMES_ROWS"] = 10

10

## 1. Get data.

In [2]:
dat_or_plot = read_csv("data/OR_PLOT.csv")
dat_or_cond = read_csv("data/OR_COND.csv")
dat_or_tree = read_csv("data/OR_TREE.csv")

Row,CN,PLT_CN,PREV_TRE_CN,INVYR,STATECD,UNITCD,COUNTYCD,PLOT,SUBP,TREE,⋯
,Int64,Int64,Int64?,Int64,Int64,Int64,Int64,Int64,Int64,Int64,⋯
1,1,64,missing,1999,41,0,5,1,1,1,⋯
2,2,64,missing,1999,41,0,5,1,1,2,⋯
3,3,64,missing,1999,41,0,5,1,1,3,⋯
4,5,64,missing,1999,41,0,5,1,1,5,⋯
5,6,64,missing,1999,41,0,5,1,1,6,⋯
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱
749789,1127606689290487,786780305290487,missing,2021,41,4,63,99829,4,146,⋯
749790,1127606690290487,786780305290487,missing,2021,41,4,63,99829,4,147,⋯
749791,1127606691290487,786780305290487,missing,2021,41,4,63,99829,4,148,⋯


## 2. Filter plots to those (a) in western Oregon (b) with at least one pair of observations.

In [3]:
dat_or_plot_less = @chain dat_or_plot begin
    @filter(LON < -120)
    @mutate(MATCH_CN = if_else(ismissing(PREV_PLT_CN), CN, PREV_PLT_CN))
    @group_by(MATCH_CN)
    @filter(n() > 1)
    @ungroup
    @select(STATECD, UNITCD, COUNTYCD, PLOT, MATCH_CN, INVYR, LON, LAT)
    @mutate(join = 1)
end

Row,STATECD,UNITCD,COUNTYCD,PLOT,MATCH_CN,INVYR,LON,LAT,join
,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Int64
1,41,3,31,86413,23884220010900,2001,-120.939,44.5214,1
2,41,0,7,65429,23905912010900,2001,-123.455,45.8468,1
3,41,3,21,59734,23930445010900,2001,-120.262,45.112,1
4,41,0,57,64502,23850638010900,2001,-123.944,45.6829,1
5,41,3,55,58461,23848826010900,2001,-120.904,45.3065,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
17120,41,0,5,96682,193209178020004,2017,-122.159,45.0668,1
17121,41,2,19,91895,193208964020004,2017,-123.815,43.8799,1
17122,41,1,39,80933,193208965020004,2017,-122.044,44.1549,1


## 3. Filter conditions to private Douglas fir.

In [4]:
dat_or_cond_less = @chain dat_or_cond begin
    @filter(FORTYPCD in 201:203,  OWNGRPCD == 40)
    #  Select columns to keep for joins. 
    @select(STATECD, UNITCD, COUNTYCD, PLOT, CONDID, CONDPROP_UNADJ, INVYR, STDAGE, SITECLCD, DSTRBCD1, DSTRBYR1, TRTCD1, TRTYR1)
    #  Select fewer columns (for now).
    @select(STATECD, UNITCD, COUNTYCD, PLOT, CONDID, CONDPROP_UNADJ, INVYR)
end

Row,STATECD,UNITCD,COUNTYCD,PLOT,CONDID,CONDPROP_UNADJ,INVYR
,Int64,Int64,Int64,Int64,Int64,Float64,Int64
1,41,4,63,97331,1,0.801911,2001
2,41,4,63,53628,1,1.0,2001
3,41,2,15,53474,1,0.615244,2001
4,41,1,41,61638,1,1.0,2001
5,41,4,63,59287,1,1.0,2001
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
2876,41,3,65,67968,1,1.0,2019
2877,41,0,7,68379,1,1.0,2019
2878,41,1,39,71081,3,0.086876,2019


## 4. Join subsets of plots and conditions.

In [5]:
dat_or_keep = @chain dat_or_cond_less begin
    @left_join(dat_or_plot_less)
    @filter(join == 1)
end

Row,STATECD,UNITCD,COUNTYCD,PLOT,CONDID,CONDPROP_UNADJ,INVYR,MATCH_CN,LON,LAT,⋯
,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Int64?,Float64?,Float64?,⋯
1,41,0,7,65429,1,0.868175,2001,23905912010900,-123.455,45.8468,⋯
2,41,0,7,65429,2,0.131825,2001,23905912010900,-123.455,45.8468,⋯
3,41,2,15,53474,1,0.615244,2001,23915019010900,-124.33,42.5568,⋯
4,41,1,41,61638,1,1.0,2001,23869938010900,-123.79,44.7156,⋯
5,41,1,39,74705,1,1.0,2001,23868974010900,-123.091,43.8213,⋯
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱
1603,41,2,15,79813,1,0.805744,2019,29880344010497,-124.334,42.2075,⋯
1604,41,0,67,52996,1,1.0,2019,29881502010497,-123.204,45.7081,⋯
1605,41,0,71,93977,2,0.914735,2019,29880276010497,-123.308,45.4068,⋯


## 5. Use the result of (4) to filter trees, then filter trees to Douglas fir.

In [6]:
dat_or_tree_less = @chain dat_or_tree begin
    # Select columns that we might use.
    @select(CN, STATECD, UNITCD, COUNTYCD, PLOT, CONDID, INVYR, SPGRPCD, VOLCFNET, TPA_UNADJ)
    # Get plot and condition information.
    @left_join(dat_or_keep)
    # Filter on plot and condition.
    @filter(join == 1)
    @select(-join)
    # Filter on species group (down to Douglas firs). This is equivalent to SPCD == 202.
    @filter(SPGRPCD == 10)
end

Row,CN,STATECD,UNITCD,COUNTYCD,PLOT,CONDID,INVYR,SPGRPCD,VOLCFNET,TPA_UNADJ,⋯
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64?,Float64?,⋯
1,23906794010900,41,0,7,56407,1,2001,10,4.60945,6.01805,⋯
2,23906798010900,41,0,7,56407,1,2001,10,3.0187,6.01805,⋯
3,23906769010900,41,0,7,56407,1,2001,10,62.3187,0.999188,⋯
4,23906777010900,41,0,7,56407,1,2001,10,2.4243,6.01805,⋯
5,23906800010900,41,0,7,56407,1,2001,10,8.33663,6.01805,⋯
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱
31911,1285676658290487,41,2,11,53140,1,2021,10,41.4217,6.01805,⋯
31912,1285676662290487,41,2,11,53140,1,2021,10,34.8015,6.01805,⋯
31913,1285676664290487,41,2,11,53140,1,2021,10,10.8199,6.01805,⋯


## 6. Aggregate to volume-per-acre, then pivot.

In [7]:
dat_or_tree_wide = @chain dat_or_tree_less begin
    @filter(!ismissing(VOLCFNET) & !ismissing(TPA_UNADJ))
    @group_by(STATECD, UNITCD, COUNTYCD, PLOT, CONDID, MATCH_CN, INVYR, LON, LAT)
    @summarize(VOLCFNET = sum(VOLCFNET * TPA_UNADJ))
    @ungroup
    @group_by(STATECD, UNITCD, COUNTYCD, PLOT, CONDID)
    @filter(n() == 2)
    @mutate(PLOT_UID = string(STATECD, "_", UNITCD, "_", COUNTYCD, "_", PLOT, "_", CONDID))
    @ungroup
    @select(PLOT_UID, MATCH_CN, LON, LAT, INVYR, VOLCFNET)
    @arrange(PLOT_UID, MATCH_CN, INVYR)
    @group_by(PLOT_UID)
    @mutate(WHICH = row_number())
    @mutate(WHICH = if_else(WHICH == 1, "First", "Second"))
    @ungroup
end

Row,PLOT_UID,MATCH_CN,LON,LAT,INVYR,VOLCFNET,WHICH
,String,Int64?,Float64?,Float64?,Int64,Float64,String
1,41_0_27_51533_1,22954279010497,-121.493,45.5487,2007,463.722,First
2,41_0_27_51533_1,22954279010497,-121.493,45.5487,2017,1508.13,Second
3,41_0_27_72243_1,24932528010900,-121.566,45.4899,2004,1630.26,First
4,41_0_27_72243_1,24932528010900,-121.566,45.4899,2014,2733.56,Second
5,41_0_27_75074_1,12876232010497,-121.692,45.5427,2005,44.9619,First
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
916,41_3_65_97674_1,12795490010497,-121.465,45.0413,2015,1578.15,Second
917,41_3_65_98082_1,25088840010900,-121.636,44.953,2004,2170.76,First
918,41_3_65_98082_1,25088840010900,-121.636,44.953,2014,4228.93,Second


Things get silly in the following section: pivot_wider in Tidier doesn't appear to support pivoting multiple columns.

In [9]:
dat_or_tree_wide_left = @chain dat_or_tree_wide begin
    @select(-VOLCFNET)
    @pivot_wider(names_from = WHICH, values_from = INVYR)
    @rename(Year_First = First, Year_Second = Second)
end

dat_or_tree_wide_right = @chain dat_or_tree_wide begin
    @select(-INVYR)
    @pivot_wider(names_from = WHICH, values_from = VOLCFNET)
    @rename(Volume_First = First, Volume_Second = Second)
end

dat_or_tree_wider = @left_join(dat_or_tree_wide_left, dat_or_tree_wide_right)

dat_or_differences = @chain dat_or_tree_wider begin
    @select(-MATCH_CN)
    @mutate(Year_Difference = Year_Second - Year_First,
            Volume_Difference = Volume_Second - Volume_First)
end

write_csv(dat_or_differences, "output/dat_or_differences.csv")

dat_or_differences

Row,PLOT_UID,LON,LAT,Year_First,Year_Second,Volume_First,Volume_Second,Year_Difference,Volume_Difference
,String,Float64?,Float64?,Int64?,Int64?,Float64?,Float64?,Int64,Float64
1,41_0_27_51533_1,-121.493,45.5487,2007,2017,463.722,1508.13,10,1044.41
2,41_0_27_72243_1,-121.566,45.4899,2004,2014,1630.26,2733.56,10,1103.3
3,41_0_27_75074_1,-121.692,45.5427,2005,2015,44.9619,446.564,10,401.602
4,41_0_47_52145_1,-122.483,44.8223,2009,2019,2110.81,4308.39,10,2197.58
5,41_0_47_59472_1,-122.615,44.9544,2001,2011,8512.12,213.922,10,-8298.2
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
456,41_3_65_89169_1,-121.643,45.104,2006,2016,2296.78,2481.73,10,184.952
457,41_3_65_97189_1,-121.57,45.0116,2002,2012,3206.03,1637.41,10,-1568.62
458,41_3_65_97674_1,-121.465,45.0413,2005,2015,1127.87,1578.15,10,450.289


## Visualization?